## Data Load

In [1]:
import sqlite3

def get_db(db_name):
	db = sqlite3.connect(db_name)
	return db

def select(db_name):
    db = get_db(db_name).cursor()
    db.execute(f"""SELECT title, date, content, link
                    FROM article;""")
    rows = db.fetchall()
    return rows

rows = select('article.db')
len(rows)

8777

## Preprocessing

In [2]:
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

def clean(text:str):
    not_used = re.compile('[^ .?!/@$%~|0-9|ㄱ-ㅣ가-힣]+')
    dup_space = re.compile('[ \t]+')  # white space duplicate
    dup_stop = re.compile('[\.]+')  # full stop duplicate

    cleaned = not_used.sub('', text.strip()) 
    cleaned = dup_space.sub(' ', cleaned)
    cleaned = dup_stop.sub('.', cleaned) 

    return cleaned

c:\Users\kidong\anaconda3\envs\milvus\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.DataFrame(rows, columns=['title', 'date', 'content', 'link'])
df['content'] = df['content'].apply(clean).apply(lambda x : x[:800])
df.head()

,title,date,content,link
0,조국당 “축하난 거부가 옹졸? ‘거부왕’ 尹이 쫄보”,20240601,조국혁신당은 자당 의원들이 윤석열 대통령의 당선 축하 난을 거부한 것에 대해 국민의...,https://www.donga.com/news/Politics/article/al...
1,"한일, 5년반 묵은 ‘초계기 갈등’ 재발 방지 합의문 전격 도출",20240601,지난해 갈등 봉합 이어 재발방지책 합의 북한 위협 억제 위해 한일 안보협력 필수 양...,https://www.donga.com/news/Politics/article/al...
2,‘강남 오피스텔 모녀 살해’ 60대 남성 구속영장 신청,20240601,서울 강남의 오피스텔에서 모녀를 살해한 혐의를 받는 60대 남성에게 경찰이 구속영장...,https://www.donga.com/news/Society/article/all...
3,"민주·조국당, ‘특검’ 주말 장외집회…與 “민생 외면 ‘생떼 정치’”",20240601,더불어민주당과 조국혁신당 등 야당이 1일 윤석열 대통령의 해병대원 특검법 거부권 행...,https://www.donga.com/news/Politics/article/al...
4,"“사람 죽였다” 자수한 30대, 숨진 채 발견",20240601,경남 창원에서 30대 남성이 또래 여성을 살해하고 숨진 채 발견됐다. 1일 창녕경찰...,https://www.donga.com/news/Society/article/all...


## Milvus Connect

In [4]:
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

QUERY_MODEL = 'solar-embedding-1-large-query'
PASSAGE_MODEL = 'solar-embedding-1-large-passage'
COLLECTION_NAME = 'Donga'  # Collection name
EMBEDDING_FIELD_NAME ='embedding'
INDEX_TYPE = "IVF_FLAT"

# 인덱스 설정
INDEX_PARAMS = {
    'metric_type': 'COSINE',     # "L2", "IP", "COSINE" 
    'index_type': INDEX_TYPE,    # "IVF_FLAT","IVF_SQ8", "IVF_PQ", "HNSW", "ANNOY"
    'params': {"nlist": 128},
}

def milvus_connect(collection_name):
    # Milvus 설정
    DIMENSION = 4096 
    MILVUS_HOST = '127.0.0.1'
    MILVUS_PORT = '19530'     

    # Milvus 연결
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    if connections:
        print("Milvus connected")
    else:
        exit()

    # 컬렉션 존재 여부 확인 및 생성
    if utility.has_collection(collection_name):
        collection = Collection(collection_name)
        print(f"Collection '{collection_name}' loaded.")
    else:
        print(f"Collection '{collection_name}' does not exist.")
        print(f"Create '{collection_name}' Collection.")
        
        field_args = [
            FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True),
            FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=300),
            FieldSchema(name='date', dtype=DataType.INT64),
            FieldSchema(name='content', dtype=DataType.VARCHAR, max_length=3000),
            FieldSchema(name='link', dtype=DataType.VARCHAR, max_length=300),
            FieldSchema(name=EMBEDDING_FIELD_NAME, dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
        ]
        
        # 스키마 정의
        schema = CollectionSchema(fields=field_args)
        
        # 컬렉션 생성
        collection = Collection(name=collection_name, schema=schema)
        
        # 인덱스 생성
        collection.create_index(field_name=EMBEDDING_FIELD_NAME, index_params=INDEX_PARAMS)
        collection.load()
        print(f"Collection '{collection_name}' created and loaded.")
    return collection

In [5]:
from openai import OpenAI
import numpy as np

collection = milvus_connect('Donga')
print(collection)
EMBEDDING_FIELD_NAME = 'embedding'
client = OpenAI(
    api_key = "{UPSTAGE_API_KEY}",
    base_url = "https://api.upstage.ai/v1/solar"
)

Milvus connected
Collection 'Donga' loaded.
<Collection>:
-------------
<name>: Donga
<description>: 
<schema>: {'auto_id': True, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 300}}, {'name': 'date', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 3000}}, {'name': 'link', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 300}}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 4096}}], 'enable_dynamic_field': False}



In [32]:
# # SELECT 구문
# field = ['id', 'title', 'date', 'content', 'link', 'embedding']
# results = collection.query(expr="", limit=1, output_fields=field)
# for result in results:
#     print(result)

{'id': 451202359389070354, 'title': '조국당 “축하난 거부가 옹졸? ‘거부왕’ 尹이 쫄보”', 'date': 20240601, 'content': '조국혁신당은 자당 의원들이 윤석열 대통령의 당선 축하 난을 거부한 것에 대해 국민의힘이 옹졸한 정치라고 비판하자 거부왕 윤 대통령이 옹졸한 정치라고 반박했다. 김보협 수석대변인은 1일 논평을 통해 의원실 앞에 몰래 난 화분을 놓고 가는 행위를 협치로 보는 국민은 없을 것이라고 비판했다. 김 수석대변인은 용산 대통령실 혹은 여당인 국민의힘 그 누구라도 조국혁신당에 만남이나 대화를 제안한 적이 있느냐며 창당한 지 석 달이 다 돼가는데도 대통령실 정무수석은 코빼기도 보이지 않는다고 지적했다. 이어 자신과 가족을 위해 거부권을 남발하는 거부왕 윤 대통령만큼 옹졸한 정치를 잘 보여주는 이는 없을 것이라고 주장했다. 김 수석대변인은 또 출입기자단 초청 대통령과의 만찬 메뉴였던 계란말이와 김치찌개에 대파가 빠졌다던데 그런 게 옹졸 쫄보의 상징이라고 비꼬았다. 조국혁신당 조국 대표는 지난달 31일 페이스북에 윤 대통령의 축하 난 사진과 함께 역대 유례없이 사익을 위하여 거부권을 오남용하는 대통령의 축하 난은 정중히 사양한다고 썼다. 윤 대통령은 지난달 30일 22대 국회 당선인 300명 전원에게 국회의원 당선을 축하합니다. 대통령 윤석열이라고 적힌 리본이 달린 난을 보냈다. 문재인 정부 국립외교원장 출신인 김준형 의원은 불통령불통대통령의 합성어의 난을 버린다고 인증했다. 김 의원은 버립니다라고 적힌 메모지를 부착한 축하 난 사진도 함께 올렸다. 같은 당 차규근 의원은 리본 가운데 대통령 윤석열이라고 적힌 부분을 가위로 잘라낸 사진을 자신의 페이스북에 올렸다. 당론 1호 법안인 한동훈 특검법을 대표 발의한 박은정 의원은 잘 ', 'link': 'https://www.donga.com/news/Politics/article/all/20240601/125225554/2', 'embedding': [-0.0

## Vector Search

In [6]:
def vector_search(collection, query, index_params, option=None):
    # Query embedding 생성
    query_embedding = client.embeddings.create(
        model="solar-embedding-1-large-query",
        input=query
    ).data[0].embedding
    query_embedding = np.asarray(query_embedding)
    
    # 공통 search 파라미터 설정
    search_params = {
        "data": [query_embedding],
        "anns_field": EMBEDDING_FIELD_NAME,
        "param": index_params,
        "limit": 5,
        "output_fields": ['title', 'date', 'content', 'link']
    }

    # 옵션이 있는 경우 expr 추가
    if option:
        search_params["expr"] = option

    # 검색 실행
    result = collection.search(**search_params)

    # 결과 처리
    result_list = []
    for hits in result:
        for hit in hits:
            result_list.append(hit.to_dict())
    
    return result_list

In [8]:
vector_search(collection, "최근 조국혁신당과 윤석열 대통령 사이의 관계를 알아볼 수 있을 만한 기사를 알려줘", INDEX_PARAMS)

[{'id': 451202359389070354,
  'distance': 0.49569398164749146,
  'entity': {'content': '조국혁신당은 자당 의원들이 윤석열 대통령의 당선 축하 난을 거부한 것에 대해 국민의힘이 옹졸한 정치라고 비판하자 거부왕 윤 대통령이 옹졸한 정치라고 반박했다. 김보협 수석대변인은 1일 논평을 통해 의원실 앞에 몰래 난 화분을 놓고 가는 행위를 협치로 보는 국민은 없을 것이라고 비판했다. 김 수석대변인은 용산 대통령실 혹은 여당인 국민의힘 그 누구라도 조국혁신당에 만남이나 대화를 제안한 적이 있느냐며 창당한 지 석 달이 다 돼가는데도 대통령실 정무수석은 코빼기도 보이지 않는다고 지적했다. 이어 자신과 가족을 위해 거부권을 남발하는 거부왕 윤 대통령만큼 옹졸한 정치를 잘 보여주는 이는 없을 것이라고 주장했다. 김 수석대변인은 또 출입기자단 초청 대통령과의 만찬 메뉴였던 계란말이와 김치찌개에 대파가 빠졌다던데 그런 게 옹졸 쫄보의 상징이라고 비꼬았다. 조국혁신당 조국 대표는 지난달 31일 페이스북에 윤 대통령의 축하 난 사진과 함께 역대 유례없이 사익을 위하여 거부권을 오남용하는 대통령의 축하 난은 정중히 사양한다고 썼다. 윤 대통령은 지난달 30일 22대 국회 당선인 300명 전원에게 국회의원 당선을 축하합니다. 대통령 윤석열이라고 적힌 리본이 달린 난을 보냈다. 문재인 정부 국립외교원장 출신인 김준형 의원은 불통령불통대통령의 합성어의 난을 버린다고 인증했다. 김 의원은 버립니다라고 적힌 메모지를 부착한 축하 난 사진도 함께 올렸다. 같은 당 차규근 의원은 리본 가운데 대통령 윤석열이라고 적힌 부분을 가위로 잘라낸 사진을 자신의 페이스북에 올렸다. 당론 1호 법안인 한동훈 특검법을 대표 발의한 박은정 의원은 잘 ',
   'link': 'https://www.donga.com/news/Politics/article/all/20240601/125225554/2',
   'title': '조국당 “축하난 거부가 

In [9]:
vector_search(collection, "최근 조국혁신당과 윤석열 대통령 사이의 관계를 알아볼 수 있을 만한 기사를 알려줘", INDEX_PARAMS, 'date >= 20240602')

[{'id': 451202359389077830,
  'distance': 0.46683454513549805,
  'entity': {'title': '조국 “맥베스와 아내의 최후 온다…아무도 尹 구하지 못한다”',
   'date': 20240621,
   'content': '조국 조국혁신당 대표가 21일 윤석열 대통령을 맥베스에 빗대며 맥베스와 그 아내의 최후가 오고 있다고 저격했다. 조 대표는 이날 페이스북에 용산과 국방부가 동시에 난리를 친 이유는 하나밖에 없다며 모든 의혹은 윤 대통령으로 모이고 있다고 적었다. 그는 만약 문재인 대통령과 청와대 비서실이 최근 언론에 보도된 것과 같이 집중적 전화질을 했다면 윤석열 검찰은 어떻게 했을까라며 압수수색 소환 구속영장 청구가 줄을 이었을 것이라고 했다. 이어 특검이 발동됐을 때 어떤 일이 전개될지 알기에 윤 대통령은 안면 몰수하고 결사 방어한다며 그러나 누군가는 입을 열 것이고 결국 둑은 무너질 것이라고 강조했다. 조 대표는 윌리엄 세익스피어 4대 비극 중 하나인 맥베스를 언급하기도 했다. 작품 속에서 스코틀랜드 국왕 맥베스는 자신이 왕이 될 것이란 예언을 듣고 자신들에게 호의를 베푼 왕을 살해하고 왕위를 찬탈한 뒤 종국에 파멸에 이른다. 조 대표는 천공도 윤 대통령의 손바닥에 왕자를 그려준 용한 사람도 그를 구해주지 못한다며 왕이 될 것이라는 세 명의 마녀의 예언을 듣고 던킨 왕을 살해하고 왕위를 박탈한 맥베스와 그 아내의 최후의 날이 오고 있다고 했다. 그는 2021년 7월에도 윤 대통령을 맥베스에 빗대어 저격했다. 당시 조 대표는 윤 대통령을 직접적으로 언급하지는 않았다. 하지만 권력욕에 휩싸인 이 두 부부는 점점 광기에 휩싸인다. 맥베스 부부의 최후? 굳이 적지 않겠다고 한 바 있다. 한편 채상병 특검법 입법을 두고 법사위는 이날 오전 10시부터 청문회를 진행 중이다. 이날 청문회에서는 이종섭 전 국방 부장관과 신범철 전 차',
   'link': 'https://www.donga.com/news/

In [7]:
vector_search(collection, "임성근 전 해병대 1사단장과 관련된 이슈를 다룬 기사를 알고 싶어.", INDEX_PARAMS)

[{'id': 451202359389085360,
  'distance': 0.5360216498374939,
  'entity': {'title': '[단독]‘도이치’ 이모 씨 “‘VIP’는 대통령 아닌 김계환, 김여사 번호도 몰라”…구명 로비 의혹 부인',
   'date': 20240710,
   'content': '도이치모터스 주가 조작 사건으로 재판을 받고 있는 이모 씨블랙펄인베스트먼트 전 대표가 임성근 전 해병대1사단장의 구명 로비 의혹에 대해 녹취록을 제보한 변호사가 의도를 가지고 내용을 퍼뜨린 것이라고 일축했다. 이 씨는 변호사가 공개한 녹취록에 담긴 도 윤석열 대통령이 아닌 김계환 해병대 사령관이라는 입장이다. 이 씨는 10일 동아일보 기자와 만나 녹취록의 언급이 마치 제가 한 이야기처럼 보도가 됐는데 같은 카카오톡 단체 채팅방에 있던 멤버 씨와 통화한 것을 변호사에게 전달해 말한 것 뿐이라고 말했다. 채 상병 순직 수사 외압 의혹을 수사 중인 고위공직자범죄수사처공수처는 이 씨가 임성근 전 해병대 1사단장을 구명했다고 주변에 자랑했다는 취지의 진술과 전화 녹음 파일을 확보한 것으로 알려졌다. 동아일보가 확보한 변호사와 이 씨의 통화 녹음파일에 따르면 이 씨는 임 전 사단장이 사표를 낸다고 그래 가지고 씨가 전화 왔더라고. 그래 가지고 내가 절대 사표 내지 마라 내가 한테 얘기를 하겠다라고 말했다. 변호사가 올해 3월 통화에서도 임 전 사단장이 채 상병 순직 사건에 책임이 있는 것 같다고 하자 이 씨는 그러니까 쓸데없이 내가 거기 개입이 돼 가지고 사표 낸다고 그럴 때 내라 그럴걸이라고 말한 것으로 확인됐다. 이에 대해 이 씨는 이날 동아일보에 채상병 사건이 일어나고 해병대 후배인 씨가 임 전 사단장이 힘들어 한다. 극단 선택할 것 같다. 나쁜 생각 말라고 이렇게 보냈는데 한 번 봐주시라며 메시지를 보내왔다며 녹취록에 언급된 상황에 대해 설명했다. 씨는 이 씨와 변호사가 함께 있는 카카오톡 단체 대화방의 멤버로 ',
   'link': 'htt

In [9]:
vector_search(collection, "임성근 전 해병대 1사단장과 관련된 이슈를 다룬 기사를 알고 싶어.", INDEX_PARAMS, "date <= 20240701")

[{'id': 451202359389079978,
  'distance': 0.49318087100982666,
  'entity': {'title': '“임성근, 혐의자서 제외”…국방부 법무관실 의견 제시',
   'date': 20240626,
   'content': '국방부 법무관리관실이 지난해 8월 해병대 채모 상병 순직 사건을 재검토하던 국방부 조사본부에 임성근 전 해병대 1사단장을 혐의자에서 제외해야 한다는 의견을 제시한 것으로 드러났다. 26일 박정훈 전 해병대 수사단장대령 측이 중앙군사법원 재판에서 확보한 법무관리관실의 변사사건 의견 요청 회신 문건에 따르면 지난해 8월 14일 법무관리관실은 임성근 1사단장의 경우 수색작전 관련 안전통제 대책을 제대로 강구하지 않는 등의 과실이 있으나 사망과의 인과관계는 명확하지 않아 경찰에서 추가 검토가 필요하다고 밝혔다. 국방부가 경찰로 사건을 이첩할 때 임 전 사단장을 인지통보서에 혐의자로 적시하지 말아야 한다는 의견을 제시한 것이다. 이후 조사본부는 임 전 사단장을 혐의자에서 제외하고 대대장 2명에게만 혐의를 적시해 경찰로 사건을 넘겼다. 법무관리실의 이같은 의견은 같은 날 조사본부가 법무관리실에 제출한 재검토 중간결과 보고서를 반박하는 취지로 풀이된다. 조사본부는 이 보고서에서 임 전 사단장에 대해 장화를 신고 수색하게 한 혐의 작전전개를 재촉한 혐의 적색티 작업 지시 혐의 등이 있다고 적시한 바 있다. 고위공직자범죄수사처는 국방부 수뇌부가 조사본부에게 사실상 임 전 사단장을 제외하라는 가이드라인을 제시한 것으로 보고 수사하고 있다. 유재은 국방부 법무관리관은 당시 임 전 사단장 등 특정인을 제외하라는 지시를 한 적이 없다는 입장이다. 구민기 기자 @.',
   'link': 'https://www.donga.com/news/Society/article/all/20240626/125642100/1'}},
 {'id': 451202359389079904,
  'distance': 0.48773944377

In [14]:
vector_search(collection, "한동훈 팬클럽들이 좋아할만한 기사를 골라줘.", INDEX_PARAMS)

[{'id': 451202359389083884,
  'distance': 0.4760681390762329,
  'entity': {'link': 'https://www.donga.com/news/Politics/article/all/20240706/125799904/2',
   'title': '한동훈 “팬덤? 나이들면 주제파악 잘해야…이재명처럼 할생각 없어”',
   'date': 20240706,
   'content': '한동훈 국민의힘 당대표 후보는 팬덤에 기댄 정치를 하지 않겠다고 말했다. 한 후보는 5일 밤 라디오 신율의 뉴스 정면 승부에서 여권 정치인 중 유일하게 팬덤을 보유하고 있다는 말에 너무 고맙다면서도 다만 저희같이 나이 드는 남자들은 주제 파악을 잘해야 된다며 제가 대단히 매력 있고 제가 너무 좋아서 모이신 건 아니라는 걸 잘 알고 있다고 했다. 그러면서 그동안 제가 가지고 있었던 포지션 스탠스 사회에 대한 입장에 공감해 모인 정도라고 생각한다며 그렇기 때무에 제가 여기서 조금만 엇나가면 얼마든지 없어질 분들이라고 생각한다. 지금은 격려와 응원의 대상이라기보다는 비판과 감시의 대상인 것도 제가 안다고 말했다. 이어 저를 좋아해 주시는 분들도 너무 고맙지만 그걸 잊지 않아주셨으면 좋겠다. 단적으로 저도 그런 마음가짐을 갖고 있다고 했다. 한 후보는 누가 팬클럽 카페 등이 10배 늘어나는 계기가 본인이 거기 가입해서 글을 남기는 경우다라고 제안하더라. 이재명 대표 같은 경우 재명이네 마을에서 그렇게 했잖냐며 정치 시작하면서 비대위원장 할 때 그런 제안을 하신 분이 있었는데 전 그건 안 하려고 한다고 못 박았다. 그 이유에 대해선 그런 식이 되면 서로 간 좋은 영향도 있겠지만 안 좋은 영향도 있을 것 같기에 그런 차원에서 안 할 것이라며 이 자리를 빌려서 대단히 고맙다는 말씀을 드리지만 저는 기본적으로는 감시와 비판의 대상이라고 강조했다. 박태근 동아닷컴 기자 @.'}},
 {'id': 451202359389082428,
  'distanc